# Fire size and severity

Generate a scatter plot of westside fires 1984-2020. Fire size on x-axis, % high severity (>75% mortality) on y-axis.

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import rasterstats
import plotly.graph_objects as go
from datetime import date

## Input data

In [ ]:
data_dir = os.path.join("..", "data")
target_crs = "EPSG:5070"
SQM_PER_HA = 10_000
today = date.today().strftime(format="%Y%m%d")

### Ecoregion

In [ ]:
westside = gpd.read_file(os.path.join(data_dir, "study_area.gpkg"))

### Fires 1984-2019

In [ ]:
# Available at https://www.mtbs.gov/direct-download
mtbs_path = os.path.join(data_dir, "mtbs", "mtbs_perims_DD.shp")
mtbs = gpd.read_file(mtbs_path).to_crs(target_crs)
mtbs_westside = gpd.clip(mtbs, westside)

# Remove prescribed fires
mtbs_westside = mtbs_westside[mtbs_westside.Incid_Type.eq("Wildfire")]
# Parse the year from the ignition date because MTBS changed their metadata to not include years
mtbs_westside["year"] = pd.DatetimeIndex(mtbs_westside.Ig_Date).year
mtbs_westside["fire_name"] = mtbs_westside.Incid_Name.str.title()

### Fires 2020

In [ ]:
# These are pre-clipped to westside ecoregions
nifc_path = os.path.join(data_dir, "NIFC", "Public_NIFS_Perimeters_westside_ecoregions_clipped.shp")
nifc_westside = gpd.read_file(nifc_path).to_crs(target_crs)
nifc_westside["fire_name"] = nifc_westside.IncidentNa.str.title()
nifc_westside["year"] = 2020

### Combined fires

In [ ]:
keep_cols = ["fire_name", "year", "geometry"]
fires = pd.concat([mtbs_westside[keep_cols], nifc_westside[keep_cols]])
fires["hectares"] = fires.area.divide(SQM_PER_HA)

### Fire severity

In [ ]:
sev_dir = os.path.join(data_dir, "severity")
sev_codes = {
    1: "Very low/unburned",
    2: "Low",
    3: "Moderate", 
    4: "Moderate high",
    5: "High", 
    6: "Very high"
}

## Processing

In [ ]:
fires["year_group"] = fires.year.apply(lambda x : "2020" if x==2020 else "1985-2019")

### Percent high severity

In [ ]:
# Calculate the percent of pixels in a list of classes from a dictionary returned by rasterstats
def percent_in_classes(d, classes=[5, 6]):
    total_count = sum(d.values())
    if not total_count:
        return 0
    
    class_count = 0
    
    for val in classes:
        try:
            class_count += d[val]
        except KeyError:
            pass

    class_percent = class_count / total_count
    return class_percent

In [ ]:
processed_fires = []

for year in fires.year.unique():
    # The severity grids can be generated using `scripts/severity.js`
    sev_path = os.path.join(sev_dir, f"severity_{year}.tif")
    yr_fires = fires.loc[fires.year.eq(year)].copy()
    
    yr_fires["severity"] = rasterstats.zonal_stats(yr_fires, sev_path, categorical=True, cmap=sev_codes, nodata=0)
    
    processed_fires.append(yr_fires)

processed_fires = pd.concat(processed_fires)
processed_fires["percent_high_severity"] = processed_fires.severity.apply(percent_in_classes)

In [ ]:
fig = go.Figure()

fires_pre = processed_fires[processed_fires.year_group.ne("2020")].copy()
fires_2020 = processed_fires[processed_fires.year_group.eq("2020")].copy()

# 1985-2019 fire points
fig.add_trace(go.Scatter(x=fires_pre.hectares, y=fires_pre.percent_high_severity, mode="markers", name="1985-2019",
                         marker=dict(size=8, symbol="0", color="rgba(55, 126, 184, 1)", 
                                     line=dict(width=1, color="rgb(255, 255, 255)"))
                        ))

# 2020 fire points
fig.add_trace(go.Scatter(x=fires_2020.hectares, y=fires_2020.percent_high_severity, 
                         mode="markers", name="2020",
                         marker=dict(size=9, symbol=2, color="rgba(255, 26, 28, 1)", 
                                     line=dict(width=1, color="rgb(0, 0, 0)"))
                        ))


fig.update_xaxes(title="Fire size (ha)")
fig.update_yaxes(title="Portion high severity", tickformat="%p")

fig.update_layout(
    template="ggplot2",
    plot_bgcolor='rgb(255, 255, 255)',
)

# Add solid plot border and grey gridlines
fig.update_yaxes(showline=True, linecolor="black", linewidth=1, mirror=True, gridcolor="rgba(0, 0, 0, 0.1)", dtick=0.2,
                 title_standoff=4
                )
fig.update_xaxes(showline=True, linecolor="black", linewidth=1, mirror=True, gridcolor="rgba(0, 0, 0, 0.1)", dtick=10_000)


fig.update_layout(
    width=830,
    height=400,
    legend=dict(title="", x=0.834, y=0.98, borderwidth=1, bordercolor="rgba(0,0,0,0.4)"),
    margin=dict(l=10, r=10, t=10, b=10)
)

fig